In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np 
import pandas as pd 
import re
import nltk 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, precision_recall_curve

In [ ]:
# Loading the data
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Tweets_data_NLP_Homework/Covid_19_tweets_train.csv',  engine='python')

# Helper Function

In [ ]:
def evaluate(model, x_test, y_test):
    prediction = model.predict(x_test)
    # prediction_prob = model.predict_proba(x_test)
    confusion = confusion_matrix(y_test, prediction)
    print("Confusion matrix",print_cm(confusion, ['Not pay', 'pay']))
    print('Accuracy: {:.2f}'.format(accuracy_score(y_test, prediction)))
    print('Precision: {:.2f}'.format(precision_score(y_test, prediction, average='macro')))
    print('Recall: {:.2f}'.format(recall_score(y_test, prediction, average='macro')))
    print('F1: {:.2f}'.format(f1_score(y_test, prediction, average='macro')))
    print(classification_report(y_test, prediction))
    # precision_recall_curve(y_test, prediction_prob)

def clean_by_pattern(text, pattern):
  '''repalcing a matched pattern in a text with a single space, and return a list of the text sentences.'''
  cleaned_data = []
  for d in text:
    cleaned_data.append(re.sub(pattern,' ',str(d)))
    
  return cleaned_data

def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    """pretty print for confusion matrixes"""
    columnwidth = max([len(x) for x in labels] + [5])  # 5 is value length
    empty_cell = " " * columnwidth
    # Print header
    print("    " + empty_cell, end=" ")
    for label in labels:
        print("%{0}s".format(columnwidth) % label, end=" ")
    print()
    # Print rows
    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % label1, end=" ")
        for j in range(len(labels)):
            cell = "%{0}.1f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            print(cell, end=" ")
        print()

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# Text Cleaning:

Cleanning using regular expression:

In [ ]:
patterns = [r"https://(\S*)", r"@(\S*)", r"[^a-zA-Z#]", r"\s\s+", r'\s+[a-zA-Z]\s+']

temp = []
cleaned_tweets = []
cleaned_tweets = clean_by_pattern(data["OriginalTweet"], patterns[0]) # remove urls and images
temp = clean_by_pattern(cleaned_tweets, patterns[1]) # remove mentions
cleaned_tweets = clean_by_pattern(temp, patterns[2]) # remove the rest(numbers, punctuations, ....) except the hashtags.
temp = clean_by_pattern(cleaned_tweets, patterns[3]) # replace multiple space (which we have after applying the previous pattern) with single space. 
cleaned_tweets = clean_by_pattern(temp, patterns[4]) # remove single characters (which we have after applying the previous pattern).
# convert all tweets characters to lower case.
temp = []
for itr in range(0, len(cleaned_tweets)):
    temp.append(cleaned_tweets[itr].lower())
cleaned_tweets = temp

Removing stopwords

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stwd = stopwords.words('english')

final_cleaned_tweets_tokens = []
for tweet in cleaned_tweets:
    sentence = [w for w in tweet.split() if w not in stwd] 
    # if sentence:
    #     final_cleaned_tweets_tokens.append(sentence)
    final_cleaned_tweets_tokens.append(sentence)
final_cleaned_tweets_tokens[:3]
final_cleaned_tweets = []
for item in final_cleaned_tweets_tokens:
    final_cleaned_tweets.append(" ".join(item))
print(final_cleaned_tweets[:2])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
['', 'advice talk neighbours family exchange phone numbers create contact list phone numbers neighbours schools employer chemist gp set online shopping accounts poss adequate supplies regular meds order']


In [ ]:
# Add the cleaned tweets to the data
final_cleaned_tweets_df = pd.DataFrame(final_cleaned_tweets)
data['Cleaned Tweets'] = final_cleaned_tweets_df 

Encoding sentiments for trainning:

In [ ]:
# Find and Replace tech 
cleanup_nums = {"Sentiment": {"Neutral": 0, "Positive": 1, "Negative": 2, "Extremely Positive": 3,
                                  "Extremely Negative": 4}}
Labeled_data = data.replace(cleanup_nums)
x = []
y = []
for key, item in data['Cleaned Tweets'].items():
  if len(item):
    x.append(item)
    y.append(Labeled_data['Sentiment'][key])

In [ ]:
x = pd.Series(x)
y = pd.Series(y)
print(x)
print(y)

0        advice talk neighbours family exchange phone n...
1        coronavirus australia woolworths give elderly ...
2        food stock one empty please panic enough food ...
3        ready go supermarket #covid outbreak paranoid ...
4        news region first confirmed covid case came su...
                               ...                        
41126    airline pilots offering stock supermarket shel...
41127    response complaint provided citing covid relat...
41128    know getting tough rationing toilet paper #cor...
41129    wrong smell hand sanitizer starting turn #coro...
41130    well new used rift going amazon rn although no...
Length: 41131, dtype: object
0        1
1        1
2        1
3        4
4        1
        ..
41126    0
41127    4
41128    1
41129    0
41130    2
Length: 41131, dtype: int64


# Splitting the data:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split


X, y = x, y
# create train and validation sets
X_tr_, X_test, y_tr_, y_test = train_test_split(X, y, test_size=0.2, random_state=1000)

# create train and validation sets from traning split
X_tr, X_val, y_tr, y_val = train_test_split(X_tr_, y_tr_, test_size=0.3, random_state=1000)
print('Train shape:', X_tr_.shape, 'Val shape:', X_test.shape)
print('Train shape:', X_tr.shape, 'Val shape:', X_val.shape)

Train shape: (32904,) Val shape: (8227,)
Train shape: (23032,) Val shape: (9872,)


# Bag-of-Word Vectorizer:

create the transformers:

In [ ]:
Bow_vectorizer = CountVectorizer()

Bow_vectorizer.fit(X_tr)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

Transforming data:

In [ ]:
# transform training data
X_tr_bow = Bow_vectorizer.transform(X_tr)

# transform validation data
X_val_bow = Bow_vectorizer.transform(X_val)

# transform test data
X_test_bow = Bow_vectorizer.transform(X_test)

# BOW models

In [ ]:
#Naive bayes
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score

NB_model_bow = GaussianNB().fit(X_tr_bow.toarray() ,y_tr)
NB_predictions_bow = NB_model_bow.predict(X_test_bow.toarray())

In [ ]:
dict = {0 : "Neutral", 1 : "Positive", 2 : "Negative", 3 : "Extremely Positive",
                                  4 : "Extremely Negative"}
                                  
evaluate(NB_model_bow, X_test_bow.toarray(), y_test)
for pre, sen in zip(NB_predictions_bow[:10], X_test[:10]):
  print(f"test :{sen}, \nresult : {dict[pre]}")

            Not pay     pay 
    Not pay   340.0   143.0 
        pay   308.0   357.0 
Confusion matrix None
Accuracy: 0.26
Precision: 0.29
Recall: 0.29
F1: 0.27
              precision    recall  f1-score   support

           0       0.32      0.23      0.27      1495
           1       0.34      0.16      0.22      2228
           2       0.30      0.22      0.25      1989
           3       0.34      0.33      0.33      1398
           4       0.17      0.50      0.25      1117

    accuracy                           0.26      8227
   macro avg       0.29      0.29      0.27      8227
weighted avg       0.30      0.26      0.26      8227

test :feeling angry going supermarket also seeing images vulnerable wrote old man sure #covid #coronavirus, 
result : Extremely Negative
test :english working suppliers restock get much food many homes possible however experiencing high demand currently apologise receive exact items ordered info please see kelsie, 
result : Extremely Negative
test

In [ ]:
# Decision Trees
from sklearn.tree import DecisionTreeClassifier 
dtree_model_bow = DecisionTreeClassifier(max_depth = 5).fit(X_tr_bow.toarray() ,y_tr) 
dtree_predictions_bow = dtree_model_bow.predict(X_test_bow.toarray()) 

In [ ]:
evaluate(NB_model_bow, X_test_bow.toarray(), y_test)
for pre, sen in zip(dtree_predictions_bow[:10], X_test[:10]):
  print(f"test :{sen}, \nresult : {dict[pre]}")

            Not pay     pay 
    Not pay   340.0   143.0 
        pay   308.0   357.0 
Confusion matrix None
Accuracy: 0.26
Precision: 0.29
Recall: 0.29
F1: 0.27
              precision    recall  f1-score   support

           0       0.32      0.23      0.27      1495
           1       0.34      0.16      0.22      2228
           2       0.30      0.22      0.25      1989
           3       0.34      0.33      0.33      1398
           4       0.17      0.50      0.25      1117

    accuracy                           0.26      8227
   macro avg       0.29      0.29      0.27      8227
weighted avg       0.30      0.26      0.26      8227

test :feeling angry going supermarket also seeing images vulnerable wrote old man sure #covid #coronavirus, 
result : Positive
test :english working suppliers restock get much food many homes possible however experiencing high demand currently apologise receive exact items ordered info please see kelsie, 
result : Positive
test :support food servi

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

OneVsRestClassifier_model_bow = OneVsRestClassifier(LinearSVC(random_state=10))
OneVsRestClassifier_model_bow.fit(X_tr_bow.toarray() ,y_tr)
OneVsRestClassifier_model_bow_predictions_bow = OneVsRestClassifier_model_bow.predict(X_test_bow.toarray())

In [ ]:
evaluate(NB_model_bow, X_test_bow.toarray(), y_test)
for pre, sen in zip(OneVsRestClassifier_model_bow_predictions_bow[:10], X_test[:10]):
  print(f"test :{sen}, \nresult : {dict[pre]}")

            Not pay     pay 
    Not pay   340.0   143.0 
        pay   308.0   357.0 
Confusion matrix None
Accuracy: 0.26
Precision: 0.29
Recall: 0.29
F1: 0.27
              precision    recall  f1-score   support

           0       0.32      0.23      0.27      1495
           1       0.34      0.16      0.22      2228
           2       0.30      0.22      0.25      1989
           3       0.34      0.33      0.33      1398
           4       0.17      0.50      0.25      1117

    accuracy                           0.26      8227
   macro avg       0.29      0.29      0.27      8227
weighted avg       0.30      0.26      0.26      8227

test :feeling angry going supermarket also seeing images vulnerable wrote old man sure #covid #coronavirus, 
result : Negative
test :english working suppliers restock get much food many homes possible however experiencing high demand currently apologise receive exact items ordered info please see kelsie, 
result : Positive
test :support food servi

In [ ]:
from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import LinearSVC

OneVsOneClassifier_model_bow = OneVsOneClassifier(LinearSVC(random_state=10))
OneVsOneClassifier_model_bow.fit(X_tr_bow.toarray() ,y_tr)
OneVsOneClassifier_model_bow_predictions_bow = OneVsOneClassifier_model_bow.predict(X_test_bow.toarray())

In [ ]:
evaluate(NB_model_bow, X_test_bow.toarray(), y_test)
for pre, sen in zip(OneVsOneClassifier_model_bow_predictions_bow[:10], X_test[:10]):
  print(f"test :{sen}, \nresult : {dict[pre]}")

            Not pay     pay 
    Not pay   340.0   143.0 
        pay   308.0   357.0 
Confusion matrix None
Accuracy: 0.26
Precision: 0.29
Recall: 0.29
F1: 0.27
              precision    recall  f1-score   support

           0       0.32      0.23      0.27      1495
           1       0.34      0.16      0.22      2228
           2       0.30      0.22      0.25      1989
           3       0.34      0.33      0.33      1398
           4       0.17      0.50      0.25      1117

    accuracy                           0.26      8227
   macro avg       0.29      0.29      0.27      8227
weighted avg       0.30      0.26      0.26      8227

test :feeling angry going supermarket also seeing images vulnerable wrote old man sure #covid #coronavirus, 
result : Negative
test :english working suppliers restock get much food many homes possible however experiencing high demand currently apologise receive exact items ordered info please see kelsie, 
result : Positive
test :support food servi

In [ ]:
from sklearn.multiclass import OutputCodeClassifier
from sklearn.svm import LinearSVC

OutputCodeClassifier_model_bow = OutputCodeClassifier(LinearSVC(random_state=1))
OutputCodeClassifier_model_bow.fit(X_tr_bow.toarray() ,y_tr)
OutputCodeClassifier_model_bow_predictions_bow = OutputCodeClassifier_model_bow.predict(X_test_bow.toarray())

In [ ]:
evaluate(NB_model_bow, X_test_bow.toarray(), y_test)
for pre, sen in zip(OutputCodeClassifier_model_bow_predictions_bow[:10], X_test[:10]):
  print(f"test :{sen}, \nresult : {dict[pre]}")

            Not pay     pay 
    Not pay   340.0   143.0 
        pay   308.0   357.0 
Confusion matrix None
Accuracy: 0.26
Precision: 0.29
Recall: 0.29
F1: 0.27
              precision    recall  f1-score   support

           0       0.32      0.23      0.27      1495
           1       0.34      0.16      0.22      2228
           2       0.30      0.22      0.25      1989
           3       0.34      0.33      0.33      1398
           4       0.17      0.50      0.25      1117

    accuracy                           0.26      8227
   macro avg       0.29      0.29      0.27      8227
weighted avg       0.30      0.26      0.26      8227

test :feeling angry going supermarket also seeing images vulnerable wrote old man sure #covid #coronavirus, 
result : Negative
test :english working suppliers restock get much food many homes possible however experiencing high demand currently apologise receive exact items ordered info please see kelsie, 
result : Positive
test :support food servi

In [ ]:
from sklearn.linear_model import LogisticRegression

logisticregression_model_bow = LogisticRegression()
logisticregression_model_bow.fit(X_tr_bow.toarray() ,y_tr)
logisticregression_model_bow_predictions = logisticregression_model_bow.predict(X_test_bow.toarray())

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
evaluate(NB_model_bow, X_test_bow.toarray(), y_test)
for pre, sen in zip(logisticregression_model_bow_predictions[:10], X_test[:10]):
  print(f"test :{sen}, \nresult : {dict[pre]}")

            Not pay     pay 
    Not pay   340.0   143.0 
        pay   308.0   357.0 
Confusion matrix None
Accuracy: 0.26
Precision: 0.29
Recall: 0.29
F1: 0.27
              precision    recall  f1-score   support

           0       0.32      0.23      0.27      1495
           1       0.34      0.16      0.22      2228
           2       0.30      0.22      0.25      1989
           3       0.34      0.33      0.33      1398
           4       0.17      0.50      0.25      1117

    accuracy                           0.26      8227
   macro avg       0.29      0.29      0.27      8227
weighted avg       0.30      0.26      0.26      8227

test :feeling angry going supermarket also seeing images vulnerable wrote old man sure #covid #coronavirus, 
result : Negative
test :english working suppliers restock get much food many homes possible however experiencing high demand currently apologise receive exact items ordered info please see kelsie, 
result : Positive
test :support food servi

# TF-IDF Vectorizer:

creating the transformer:

In [ ]:
Tfidf_vectorizer = TfidfVectorizer()
Tfidf_vectorizer.fit(X_tr)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

Transforming data:

In [ ]:
# transform training data
X_tr_tf = Tfidf_vectorizer.transform(X_tr)

# transform validation data
X_val_tf = Tfidf_vectorizer.transform(X_val)

# transform test data
X_test_tf = Tfidf_vectorizer.transform(X_test)

# TF_IDF Model

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score

NB_model_tf = GaussianNB().fit(X_tr_tf.toarray() ,y_tr)
NB_model_tf_predictions_tf = NB_model_tf.predict(X_test_tf.toarray())

In [ ]:
evaluate(NB_model_bow, X_test_tf.toarray(), y_test)
for pre, sen in zip(NB_model_tf_predictions_tf[:10], X_test[:10]):
  print(f"test :{sen}, \nresult : {dict[pre]}")

            Not pay     pay 
    Not pay   336.0   162.0 
        pay   368.0   396.0 
Confusion matrix None
Accuracy: 0.25
Precision: 0.28
Recall: 0.27
F1: 0.26
              precision    recall  f1-score   support

           0       0.28      0.22      0.25      1495
           1       0.33      0.18      0.23      2228
           2       0.29      0.22      0.25      1989
           3       0.32      0.28      0.30      1398
           4       0.17      0.46      0.24      1117

    accuracy                           0.25      8227
   macro avg       0.28      0.27      0.26      8227
weighted avg       0.29      0.25      0.25      8227

test :feeling angry going supermarket also seeing images vulnerable wrote old man sure #covid #coronavirus, 
result : Extremely Negative
test :english working suppliers restock get much food many homes possible however experiencing high demand currently apologise receive exact items ordered info please see kelsie, 
result : Extremely Negative
test

In [ ]:
# Decision Trees
from sklearn.tree import DecisionTreeClassifier 
dtree_model_tf = DecisionTreeClassifier(max_depth = 5).fit(X_tr_tf.toarray() ,y_tr) 
dtree_predictions_tf = dtree_model_tf.predict(X_test_tf.toarray()) 

In [ ]:
evaluate(NB_model_bow, X_test_tf.toarray(), y_test)
for pre, sen in zip(dtree_predictions_tf[:10], X_test[:10]):
  print(f"test :{sen}, \nresult : {dict[pre]}")

            Not pay     pay 
    Not pay   336.0   162.0 
        pay   368.0   396.0 
Confusion matrix None
Accuracy: 0.25
Precision: 0.28
Recall: 0.27
F1: 0.26
              precision    recall  f1-score   support

           0       0.28      0.22      0.25      1495
           1       0.33      0.18      0.23      2228
           2       0.29      0.22      0.25      1989
           3       0.32      0.28      0.30      1398
           4       0.17      0.46      0.24      1117

    accuracy                           0.25      8227
   macro avg       0.28      0.27      0.26      8227
weighted avg       0.29      0.25      0.25      8227

test :feeling angry going supermarket also seeing images vulnerable wrote old man sure #covid #coronavirus, 
result : Positive
test :english working suppliers restock get much food many homes possible however experiencing high demand currently apologise receive exact items ordered info please see kelsie, 
result : Positive
test :support food servi

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

OneVsRestClassifier_model_tf = OneVsRestClassifier(LinearSVC(random_state=1))
OneVsRestClassifier_model_tf.fit(X_tr_tf.toarray() ,y_tr)
OneVsRestClassifier_model_bow_predictions_tf = OneVsRestClassifier_model_tf.predict(X_test_tf.toarray())

In [ ]:
evaluate(NB_model_bow, X_test_tf.toarray(), y_test)
for pre, sen in zip(OneVsRestClassifier_model_bow_predictions_tf[:10], X_test[:10]):
  print(f"test :{sen}, \nresult : {dict[pre]}")

            Not pay     pay 
    Not pay   336.0   162.0 
        pay   368.0   396.0 
Confusion matrix None
Accuracy: 0.25
Precision: 0.28
Recall: 0.27
F1: 0.26
              precision    recall  f1-score   support

           0       0.28      0.22      0.25      1495
           1       0.33      0.18      0.23      2228
           2       0.29      0.22      0.25      1989
           3       0.32      0.28      0.30      1398
           4       0.17      0.46      0.24      1117

    accuracy                           0.25      8227
   macro avg       0.28      0.27      0.26      8227
weighted avg       0.29      0.25      0.25      8227

test :feeling angry going supermarket also seeing images vulnerable wrote old man sure #covid #coronavirus, 
result : Negative
test :english working suppliers restock get much food many homes possible however experiencing high demand currently apologise receive exact items ordered info please see kelsie, 
result : Positive
test :support food servi

In [ ]:
from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import LinearSVC

OneVsOneClassifier_model_tf = OneVsOneClassifier(LinearSVC(random_state=1))
OneVsOneClassifier_model_tf.fit(X_tr_tf.toarray() ,y_tr)
OneVsOneClassifier_model_tf_predictions = OneVsOneClassifier_model_tf.predict(X_test_tf.toarray())

In [ ]:
evaluate(NB_model_bow, X_test_tf.toarray(), y_test)
for pre, sen in zip(OneVsOneClassifier_model_tf_predictions[:10], X_test[:10]):
  print(f"test :{sen}, \nresult : {dict[pre]}")

            Not pay     pay 
    Not pay   336.0   162.0 
        pay   368.0   396.0 
Confusion matrix None
Accuracy: 0.25
Precision: 0.28
Recall: 0.27
F1: 0.26
              precision    recall  f1-score   support

           0       0.28      0.22      0.25      1495
           1       0.33      0.18      0.23      2228
           2       0.29      0.22      0.25      1989
           3       0.32      0.28      0.30      1398
           4       0.17      0.46      0.24      1117

    accuracy                           0.25      8227
   macro avg       0.28      0.27      0.26      8227
weighted avg       0.29      0.25      0.25      8227

test :feeling angry going supermarket also seeing images vulnerable wrote old man sure #covid #coronavirus, 
result : Negative
test :english working suppliers restock get much food many homes possible however experiencing high demand currently apologise receive exact items ordered info please see kelsie, 
result : Positive
test :support food servi

In [ ]:
from sklearn.linear_model import LogisticRegression

logisticregression_model_tf = LogisticRegression()
logisticregression_model_tf.fit(X_tr_tf.toarray() ,y_tr)
logisticregression_model_tf_predictions = logisticregression_model_tf.predict(X_test_tf.toarray())

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
evaluate(NB_model_bow, X_test_tf.toarray(), y_test)
for pre, sen in zip(logisticregression_model_tf_predictions[:10], X_test[:10]):
  print(f"test :{sen}, \nresult : {dict[pre]}")

            Not pay     pay 
    Not pay   336.0   162.0 
        pay   368.0   396.0 
Confusion matrix None
Accuracy: 0.25
Precision: 0.28
Recall: 0.27
F1: 0.26
              precision    recall  f1-score   support

           0       0.28      0.22      0.25      1495
           1       0.33      0.18      0.23      2228
           2       0.29      0.22      0.25      1989
           3       0.32      0.28      0.30      1398
           4       0.17      0.46      0.24      1117

    accuracy                           0.25      8227
   macro avg       0.28      0.27      0.26      8227
weighted avg       0.29      0.25      0.25      8227

test :feeling angry going supermarket also seeing images vulnerable wrote old man sure #covid #coronavirus, 
result : Negative
test :english working suppliers restock get much food many homes possible however experiencing high demand currently apologise receive exact items ordered info please see kelsie, 
result : Positive
test :support food servi

# Deeplearning

In [ ]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_tr)

In [ ]:
X_tr = tokenizer.texts_to_sequences(X_tr)
X_val = tokenizer.texts_to_sequences(X_val)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

In [ ]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 100

X_tr = pad_sequences(X_tr, padding='post', maxlen=maxlen)
X_val = pad_sequences(X_val, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

print(X_tr[0,:])

[   5 1071  361   45 1071   40  735 1676 1533 1242  609 2219 1676  735
  178  637 1317  153   71 2039   49 1044  204   74   40 1173    2    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]


In [ ]:
from keras.models import Sequential
from keras import layers
from keras.layers import LSTM, Dense, GRU, Embedding, Bidirectional, Dropout

Embedding_dim = 100

model = Sequential()
model.add(layers.Embedding(vocab_size, Embedding_dim, input_length=maxlen))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(Embedding_dim)))
model.add(Dense(100, activation='relu'))
model.add(Dense(5, activation='softmax'))


model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy', precision_m, recall_m, f1_m])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 100)          3151200   
_________________________________________________________________
dropout (Dropout)            (None, 100, 100)          0         
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               160800    
_________________________________________________________________
dense (Dense)                (None, 100)               20100     
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 505       
Total params: 3,332,605
Trainable params: 3,332,605
Non-trainable params: 0
_________________________________________________________________


In [ ]:
import pickle

with open('/content/drive/MyDrive/Colab Notebooks/NLP Homework/tokenizer.pkl', 'wb') as f:
  pickle.dump(tokenizer, f)

In [ ]:
from keras.callbacks import ModelCheckpoint
from keras import backend as K

filepath = "/content/drive/MyDrive/Colab Notebooks/NLP Homework/LanguageModel.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
desired_callbacks = [checkpoint]
history = model.fit(X_tr, np.array(y_tr),
                    epochs=5,
                    verbose=True,
                    validation_data=(X_val, np.array(y_val)),
                    batch_size=10,
                    callbacks=desired_callbacks)

# plot_history(history)

Epoch 1/5
2304/2304 [==============================] - 358s 145ms/step - loss: 1.3310 - accuracy: 0.4161 - precision_m: 61042162.9406 - recall_m: 1.5699 - f1_m: 2.6212 - val_loss: 0.8573 - val_accuracy: 0.6837 - val_precision_m: 1.4139 - val_recall_m: 1.3640 - val_f1_m: 1.3661

Epoch 00001: loss improved from inf to 1.13366, saving model to /content/drive/MyDrive/Colab Notebooks/NLP Homework/LanguageModel.hdf5
Epoch 2/5
2304/2304 [==============================] - 327s 142ms/step - loss: 0.7631 - accuracy: 0.7260 - precision_m: 1.2012 - recall_m: 1.3021 - f1_m: 1.2372 - val_loss: 0.8092 - val_accuracy: 0.7050 - val_precision_m: 1.1759 - val_recall_m: 1.2812 - val_f1_m: 1.2152

Epoch 00002: loss improved from 1.13366 to 0.76061, saving model to /content/drive/MyDrive/Colab Notebooks/NLP Homework/LanguageModel.hdf5
Epoch 3/5
2304/2304 [==============================] - 330s 143ms/step - loss: 0.6591 - accuracy: 0.7697 - precision_m: 1.0884 - recall_m: 1.2284 - f1_m: 1.1441 - val_loss: 0.

ValueError: ignored

In [ ]:
metrics = model.evaluate(X_tr, y_tr)
print("Train: {}".format(metrics))
metrics = model.evaluate(X_test, np.array(y_test))
print("Test:  {}".format(metrics))

Train: [0.8346096277236938, 0.72857666015625, 1.0305896997451782, 1.1803340911865234, 1.0979121923446655]
258/258 [==============================] - 13s 49ms/step - loss: 0.8346 - accuracy: 0.7286 - precision_m: 1.0306 - recall_m: 1.1803 - f1_m: 1.0979
Test:  [0.8346096277236938, 0.72857666015625, 1.0305896997451782, 1.1803340911865234, 1.0979121923446655]


In [ ]:
model.save("/content/drive/MyDrive/Colab Notebooks/NLP Homework/language_model_classification.h5")

In [ ]:
from keras.models import load_model

my_model = load_model("/content/drive/MyDrive/Colab Notebooks/NLP Homework/language_model_classification.h5")
history = my_model.fit(X_tr, np.array(y_tr),
                    epochs=10,
                    verbose=True,
                    validation_data=(X_test, np.array(y_test)),
                    batch_size=10)